In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer

from sklearn.feature_extraction.text import CountVectorizer

Using TensorFlow backend.


In [2]:
reviews = pd.read_csv('Reviews.csv')

In [3]:
print (reviews)


         Timestamp  How would you score the class   \
0    2/15/19 13:42                               1   
1    2/15/19 14:37                               1   
2    2/15/19 11:06                               2   
3    2/15/19 11:08                               2   
4    2/15/19 11:06                               3   
5    2/15/19 11:06                               3   
6    2/15/19 11:07                               3   
7    2/15/19 11:07                               3   
8    2/15/19 11:09                               3   
9    2/15/19 11:10                               3   
10   2/15/19 11:56                               3   
11   2/15/19 13:07                               3   
12   2/15/19 13:40                               3   
13   2/15/19 13:40                               3   
14   2/15/19 14:38                               3   
15   2/15/19 15:30                               3   
16   2/15/19 15:31                               3   
17   2/15/19 11:06          

In [4]:
reviews.head()

,Timestamp,How would you score the class,Explain why you gave that score? (What is good about the class? What is bad about the class? What could make the class better?),"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data
0,2/15/19 13:42,1,we suck. ...,NaN,NaN,NaN,NaN,NaN
1,2/15/19 14:37,1,no naptime / cheese ohnononononnononononononon...,NaN,NaN,NaN,NaN,NaN
2,2/15/19 11:06,2,Because of mr liberman and i need to calm down...,NaN,NaN,NaN,NaN,NaN
3,2/15/19 11:08,2,I gave this class a 2 because it is too fast f...,NaN,NaN,NaN,NaN,NaN
4,2/15/19 11:06,3,Mr. Liberman is a fun teacher but we have to r...,NaN,NaN,NaN,NaN,NaN


In [5]:
reviews.shape


(158, 8)

In [6]:
reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 8 columns):
Timestamp                                                                                                                          133 non-null object
How would you score the class                                                                                                      158 non-null int64
Explain why you gave that score? (What is good about the class? What is bad about the class? What could make the class better?)    158 non-null object
Polarity (-1, 0, 1)                                                                                                                0 non-null float64
Suggestions (0 or 1)                                                                                                               0 non-null float64
Suggestions                                                                                                                        0 non-null fl

In [7]:
reviews.describe()


,How would you score the class,"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data
count,158.000000,0.0,0.0,0.0,0.0,0.0
mean,4.044304,NaN,NaN,NaN,NaN,NaN
std,0.946633,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN
25%,4.000000,NaN,NaN,NaN,NaN,NaN
50%,4.000000,NaN,NaN,NaN,NaN,NaN
75%,5.000000,NaN,NaN,NaN,NaN,NaN
max,5.000000,NaN,NaN,NaN,NaN,NaN


In [8]:
reviews.rename(
    columns={
        'Explain why you gave that score? (What is good about the class? What is bad about the class? What could make the class better?)' : 'rev', 'How would you score the class ' : 'stars'
    },
    inplace=True
)
reviews.head()

,Timestamp,stars,rev,"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data
0,2/15/19 13:42,1,we suck. ...,NaN,NaN,NaN,NaN,NaN
1,2/15/19 14:37,1,no naptime / cheese ohnononononnononononononon...,NaN,NaN,NaN,NaN,NaN
2,2/15/19 11:06,2,Because of mr liberman and i need to calm down...,NaN,NaN,NaN,NaN,NaN
3,2/15/19 11:08,2,I gave this class a 2 because it is too fast f...,NaN,NaN,NaN,NaN,NaN
4,2/15/19 11:06,3,Mr. Liberman is a fun teacher but we have to r...,NaN,NaN,NaN,NaN,NaN


In [9]:
reviews['rev'] = reviews['rev'].apply(str)
## This converts the column values of reviews to strings

In [10]:
reviews['text length'] = reviews['rev'].apply(len)
reviews.head()

#this creates a new column that will apply the text length of review

,Timestamp,stars,rev,"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data,text length
0,2/15/19 13:42,1,we suck. ...,NaN,NaN,NaN,NaN,NaN,167
1,2/15/19 14:37,1,no naptime / cheese ohnononononnononononononon...,NaN,NaN,NaN,NaN,NaN,86
2,2/15/19 11:06,2,Because of mr liberman and i need to calm down...,NaN,NaN,NaN,NaN,NaN,117
3,2/15/19 11:08,2,I gave this class a 2 because it is too fast f...,NaN,NaN,NaN,NaN,NaN,268
4,2/15/19 11:06,3,Mr. Liberman is a fun teacher but we have to r...,NaN,NaN,NaN,NaN,NaN,113


In [11]:
X = reviews['rev']
y = reviews['stars']

In [12]:
print(type(X))

<class 'pandas.core.series.Series'>


In [13]:
# tokenizer = Tokenizer(num_words=2000)
# tokenizer.fit_on_texts(X)
# sequences = tokenizer.texts_to_sequences(X)
# one_hot_results = tokenizer.texts_to_matrix(X, mode='binary')
# word_index = tokenizer.word_index

# print('Found %s unique tokens.' % len(word_index))
# print(tokenizer)
# print(one_hot_results)
# print(word_index)


In [14]:
import re 
def getwordlist(text):
    # Convert words to lower case and split them
    text = text.lower().split()    
    text = " ".join(text)    
    #Remove Special Characters
    text=re.sub(r'[^a-z\d ]','',text)    
    #Replace Numbers
    text=re.sub(r'\d+','n',text)
    # Return a list of words
    return(text)

comments = []
for text in X:
    comments.append(getwordlist(text))

In [15]:
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(comments) 
sequences = tokenizer.texts_to_sequences(comments)


word_index = tokenizer.word_index
print('unique words %s' % len(word_index))

data = pad_sequences(sequences, maxlen=2500, padding='post')
print('Train data shape:', data.shape)



unique words 693
Train data shape: (158, 2500)


In [16]:
# test_comments = []
# for t in text:
#     test_comments.append(getwordlist(t))
    
# test_sequences = tokenizer.texts_to_sequences(test_comments)
# test_data = pad_sequences(test_sequences, maxlen=2500, padding='post')
# print('Test data shape:', test_data.shape)

In [17]:
 from sklearn.preprocessing import LabelBinarizer
 encoder = LabelBinarizer()
 transfomed_label = encoder.fit_transform(y)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, transfomed_label, test_size=0.2, random_state=42)

In [19]:
#Glove Vectors loaded

embeddings_index = {}

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:

    values = line.split()

    word = values[0]

    coefs = np.asarray(values[1:], dtype='float32')

    embeddings_index[word] = coefs

f.close()

 

print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [20]:
#do not need to do pipelining

#want to use the model.fit- parse the train data and then specify the epochs and batch size.


# prepare embeddings for layer
from keras.layers.embeddings import Embedding

words_max = len(word_index)+1

embedding_matrix = np.zeros((len(word_index)+1, 100))

for word, i in word_index.items():

    if i >= words_max:

        continue

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:

        embedding_matrix[i] = embedding_vector

In [21]:
##this is where add Karuna Code
# Create an non re-trainable embedding layer with embedding size 100

nb_words = len(word_index)+1

embedding_layer = Embedding(nb_words,100,

        weights=[embedding_matrix],

        input_length=2500,

        trainable=True)

In [22]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(embedding_layer)
    model.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7))
 
    #model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
   # model.add(LSTM(64, recurrent_dropout=0.4))
    #model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    model.summary()

    return model

## Sets up the model for how much data is processed at once.

In [23]:
model = get_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2500, 100)         69400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 111,965
Trainable params: 111,965
Non-trainable params: 0
_________________________________________________________________


In [24]:
#x is the comment, or reviews, can be many columns,  y is the labels

In [25]:
#This is the training you need to save the data to train
hist = model.fit(X_train, y_train, 
        epochs=2, batch_size=128, validation_split=0.2)


Instructions for updating:
Use tf.cast instead.
Train on 100 samples, validate on 26 samples
Epoch 1/2
100/100 [==============================] - 9s 91ms/step - loss: 1.6104 - acc: 0.0200 - val_loss: 1.5946 - val_acc: 0.4231
Epoch 2/2
100/100 [==============================] - 6s 60ms/step - loss: 1.5944 - acc: 0.3600 - val_loss: 1.5816 - val_acc: 0.4231


In [26]:
#loss = model.evaluate(X_test, y_test, verbose=1)
#print('Accuracy: %f' % (accuracy*100))
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

32/32 [==============================] - 1s 17ms/step
Test set
  Loss: 1.581
  Accuracy: 0.375


In [27]:
text = "this is a test to see if it works, Karuna is the best!!!"

In [28]:
prediction = model.predict(test_data)

NameError: name 'test_data' is not defined

In [29]:
from sklearn.metrics import accuracy_score
prediction = model.predict(X_test)
prediction = np.argmax(prediction, axis = 1)
accuracy_score(y_test, prediction)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets